In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 9

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((280830, 9), 0)

### Unobserved Nodes = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 144000
Node 0
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 11.00879504635314 7.772793263667448 0.25142900372765753 0.659033748534849 3.94944429397583
Node 1
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 13.44887816097216 9.978754071302863 0.3205404511624257 0.583641153296564 3.687145233154297
Node 2
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.096 )
( 12.112873572878746 9.403821001567607 0.3383667926090826 0.6272416106475702 3.893643617630005
Node 3
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.710952129358894 31.014904904904895 1.0 0.0 0.012948274612426758
Node 4
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.005 )
( 16.602768363071924 12.93213586496207 0.38286114961964934 0.4933663565319828 4.2047295570373535
Node 5
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.019 )
( 13.593867096103622 9.84779152496666 0.2833736338001243 0.5858874527416509 5.162204027175903
Node 6
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.79937643911535 31.043873873873874 1.0 

( 13.248889678160463 10.66300705310276 0.42002222822759566 0.599071456510937 4.684529542922974
Node 57
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.007 )
( 11.687795472184094 8.584593299251534 0.29495755571830135 0.646910905160211 4.677450895309448
Node 58
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.037 )
( 15.745457257126677 13.313479837301816 0.5367957535186636 0.5242356129833974 4.384283542633057
Node 59
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 11.926683316442777 9.231334640932898 0.3540439782014514 0.6389747806609654 4.836074590682983
Node 60
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.052 )
( 11.436450741607654 8.825940455518738 0.34243602479325563 0.6534210566854605 4.355358362197876
Node 61
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.062 )
( 15.728276460980426 12.332772417400008 0.356904307394935 0.5207674014193059 4.150908946990967
Node 62
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.003 )
( 11.191114415641575 8.512710228827936 0.3228658197532361 0.6589468889571715 4

( 9.607837110677309 6.963185813864321 0.3088826332542267 0.6398419623596217 4.337422609329224
Node 112
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 9.920102338883943 7.485684067717248 0.36001547341575 0.628905621301862 4.609664440155029
Node 113
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 11.008931760657797 8.332846177960342 0.3652965591688283 0.5874331015166379 3.9793667793273926
Node 114
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.04 )
( 9.207109602295144 6.7560164438374875 0.3344353805041866 0.6548557526277855 5.583078622817993
Node 115
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 10.340618049086881 7.769600237171435 0.3780780146802873 0.6098679850739375 5.148289203643799
Node 116
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.049 )
( 11.875535226817359 9.712056165115452 0.5270317649750292 0.5531029519026809 4.577715635299683
Node 117
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.042 )
( 20.8993385068786 19.388624103148246 1.0362256113404709 0.21290885126598047 4.4

( 17.63074232781005 14.60960164871344 0.7647583988459928 0.3285763712380878 3.287212610244751
Node 167
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.059 )
( 17.79038076113053 14.485233209051469 0.8054546975875366 0.32074346976587087 3.598383903503418
Node 168
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 29.031761667525963 27.349629860818514 1.4609562941342231 -0.10337818475828664 4.905888795852661
Node 169
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.008 )
( 16.014866917981127 13.163457289147814 0.6470670961388605 0.3912046626815656 2.889277219772339
Node 170
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.022 )
( 10.358581895063711 7.61084378028328 0.360103802678583 0.6039340361989882 4.400239706039429
Node 171
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.006 )
( 12.814999956342115 10.556821559507043 0.5742960232995836 0.5094214101275393 4.4900429248809814
Node 172
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.018 )
( 13.652004212171278 10.864106424923582 0.4984263122134218 0.4750794768

( 12.812599382177378 10.242096128245702 0.456613689163892 0.560615160781428 4.249640941619873
Node 221
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.016 )
( 10.478263659839774 8.079355824083338 0.3569211805697576 0.6419717234133249 4.848411798477173
Node 222
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.016 )
( 10.211636829291757 7.226835439974997 0.27512683350456757 0.6507602048100007 4.401181697845459
Node 223
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.024 )
( 10.662860809467945 7.833127059490489 0.31041074604186814 0.634980907291139 4.835636615753174
Node 224
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 29.15931810025866 27.487835835835845 1.0 0.0 0.011967897415161133
Node 225
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.031 )
( 10.548679596475989 7.810133490046098 0.31900020493279324 0.6389365422822514 4.8041603565216064
Node 226
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.009 )
( 11.051528785136739 7.908958543923025 0.28987084373032246 0.620842957802386 4.785212993621826
Node 2

( 15.982534691977905 12.265542055940434 0.3484215016519858 0.5309986692470254 4.4500977993011475
Node 276
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 34.09061554105423 32.22714114114114 1.0 0.0 0.010935306549072266
Node 277
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.131 )
( 18.15327471405322 14.587166430122485 0.40851420396616267 0.4686354786943805 4.790199041366577
Node 278
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.051 )
( 14.78296856473793 11.076848374325754 0.31921639056619855 0.5677623176553893 5.574920415878296
Node 279
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.044 )
( 15.837674425828505 12.093672247735288 0.34562075537705383 0.5378649117311112 5.690790176391602
Node 280
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 13.236263685695631 9.689132706066365 0.2967939079573046 0.6153389048530544 6.300161361694336
Node 281
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.051 )
( 15.625096731803703 11.874648865009378 0.34167070986601283 0.5463337917868643 5.844019889831543
Nod

( 12.361441567088521 9.923929994056989 0.4071731640015793 0.6220221313660099 4.010331392288208
Node 332
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 12.056811226271963 9.207792763679201 0.34363149968173395 0.630717280510168 4.224658250808716
Node 333
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.098 )
( 13.141788071853709 10.945283740813096 0.45951641203267773 0.5942388999016239 4.852082014083862
Node 334
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.317742801745915 30.616110110110114 1.0 0.0 0.017903804779052734
Node 335
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.052 )
( 11.455437638088418 8.370022595804823 0.29655679462037043 0.6443200632904562 4.744351387023926
Node 336
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 12.191256611978893 10.006184621286717 0.4173526978670997 0.6195413040134992 5.601029872894287
Node 337
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 31.979539441694634 30.323331331331328 1.0 0.0 0.013962268829345703
Node 338
( (2283, 12) (2283, 9) 

( 10.885669095602678 8.73760525965185 0.478018743618587 0.5827481558361649 4.7503368854522705
Node 387
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.031 )
( 12.202324175106146 9.914973125444112 0.5354262767168201 0.5317826342041807 3.8426976203918457
Node 388
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.021 )
( 11.676049831832682 9.041345244980175 0.4538824880944192 0.5491978513244324 3.6792163848876953
Node 389
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.002 )
( 23.128983292847725 21.375760479640224 1.210685820680692 0.11166917655731534 3.8626270294189453
Node 390
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 25.9236960099214 23.878458458458454 1.0 0.0 0.008976221084594727
Node 391
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.018 )
( 10.875748170023297 7.673170913518811 0.3435904068929063 0.578630373500777 4.598742961883545
Node 392
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.005 )
( 10.742740916726591 7.597374340057243 0.35873832214194235 0.584218925311069 5.130302906036377
Node 3

( 15.107938616128784 12.998778200352788 0.7807671102748694 0.4019696491914758 4.320454120635986
Node 441
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.015 )
( 11.714946995185283 9.77737699256038 0.5933902192589974 0.536491601129736 4.557819128036499
Node 442
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.056 )
( 12.555318861985233 10.590397921877221 0.6347145225822589 0.5040949457233348 3.616334915161133
Node 443
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.006 )
( 10.359227330608226 7.485757772764474 0.3641955579396712 0.5908254405162148 4.412230491638184
Node 444
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.038 )
( 10.818750734559833 8.009736565751982 0.39208184295047066 0.5722109614105144 4.561788320541382
Node 445
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.036 )
( 13.745313431472578 10.445975379441819 0.4455978270544687 0.45649048328352104 3.151602029800415
Node 446
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 11.170095917518161 8.797913182161194 0.48869101921062885 0.56047830997

( 11.63736562971134 8.520503164715684 0.29967826643686846 0.6074018010560471 4.961949110031128
Node 495
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.001 )
( 10.49701791605761 7.689854596226613 0.30241134296490124 0.6472275083796467 4.429702520370483
Node 496
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 9.850213322291253 6.960744969317274 0.260437622922783 0.6697636312333832 4.839901447296143
Node 497
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.039 )
( 10.189368822146555 7.218680750895341 0.26610586980090734 0.6597826913007709 4.178862810134888
Node 498
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.025 )
( 11.05572837204718 8.762405700701356 0.368844276156927 0.6347267823149361 5.150301694869995
Node 499
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 10.134146656342047 7.318321940223661 0.27944491443757996 0.666741287358777 3.969407796859741
Node 500
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 30.502795232519215 28.842562562562573 1.0 0.0 0.01196908950805664
Node 501
( 

### Unobserved Nodes = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 290880
Node 0
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 11.00879504635314 7.772793263667448 0.25142900372765753 0.659033748534849 4.507953882217407
Node 1
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 13.44887816097216 9.978754071302863 0.3205404511624257 0.583641153296564 3.956979751586914
Node 2
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.096 )
( 12.112873572878746 9.403821001567607 0.3383667926090826 0.6272416106475702 3.9294698238372803
Node 3
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.710952129358894 31.014904904904895 1.0 0.0 0.010971546173095703
Node 4
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.005 )
( 16.602768363071924 12.93213586496207 0.38286114961964934 0.4933663565319828 4.25966739654541
Node 5
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.019 )
( 13.593867096103622 9.84779152496666 0.2833736338001243 0.5858874527416509 5.080307245254517
Node 6
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.013 )
( 11.926909577803368 8.368485427120191 0.

( 10.74000064393313 7.724924876898261 0.2608809003178196 0.6758491491887426 5.186138153076172
Node 56
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 13.248889678160463 10.66300705310276 0.42002222822759566 0.599071456510937 4.083077430725098
Node 57
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.007 )
( 11.687795472184094 8.584593299251534 0.29495755571830135 0.646910905160211 4.116966247558594
Node 58
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.037 )
( 15.745457257126677 13.313479837301816 0.5367957535186636 0.5242356129833974 3.7898755073547363
Node 59
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 11.926683316442777 9.231334640932898 0.3540439782014514 0.6389747806609654 4.324480295181274
Node 60
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.99811186517596 31.26377977977978 1.0 0.0 0.009972333908081055
Node 61
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.062 )
( 15.728276460980426 12.332772417400008 0.356904307394935 0.5207674014193059 3.714063882827759
Node 62
( (2283,

( 9.607837110677309 6.963185813864321 0.3088826332542267 0.6398419623596217 4.043183088302612
Node 112
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 26.73201996129756 25.200436436436437 1.0 0.0 0.009973526000976562
Node 113
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 11.008931760657797 8.332846177960342 0.3652965591688283 0.5874331015166379 3.6602377891540527
Node 114
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.04 )
( 9.207109602295144 6.7560164438374875 0.3344353805041866 0.6548557526277855 5.119267463684082
Node 115
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 10.340618049086881 7.769600237171435 0.3780780146802873 0.6098679850739375 4.8251423835754395
Node 116
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.049 )
( 11.875535226817359 9.712056165115452 0.5270317649750292 0.5531029519026809 4.225700378417969
Node 117
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.042 )
( 20.8993385068786 19.388624103148246 1.0362256113404709 0.21290885126598047 3.984351873397827
Node 118


( 17.79038076113053 14.485233209051469 0.8054546975875366 0.32074346976587087 3.020958423614502
Node 168
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 29.031761667525963 27.349629860818514 1.4609562941342231 -0.10337818475828664 4.47601842880249
Node 169
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.008 )
( 16.014866917981127 13.163457289147814 0.6470670961388605 0.3912046626815656 2.7247314453125
Node 170
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.022 )
( 10.358581895063711 7.61084378028328 0.360103802678583 0.6039340361989882 4.0232343673706055
Node 171
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.006 )
( 12.814999956342115 10.556821559507043 0.5742960232995836 0.5094214101275393 4.013274431228638
Node 172
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.018 )
( 13.652004212171278 10.864106424923582 0.4984263122134218 0.47507947687690766 3.7998766899108887
Node 173
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.031 )
( 10.944781992921394 8.596612615147425 0.45048973830508793 0.576862932

( 10.478263659839774 8.079355824083338 0.3569211805697576 0.6419717234133249 4.172816038131714
Node 222
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.016 )
( 10.211636829291757 7.226835439974997 0.27512683350456757 0.6507602048100007 4.102067470550537
Node 223
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.024 )
( 10.662860809467945 7.833127059490489 0.31041074604186814 0.634980907291139 4.269587755203247
Node 224
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 29.15931810025866 27.487835835835845 1.0 0.0 0.010940074920654297
Node 225
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 29.215583496467072 27.513065065065074 1.0 0.0 0.011970281600952148
Node 226
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.009 )
( 11.051528785136739 7.908958543923025 0.28987084373032246 0.620842957802386 4.453097581863403
Node 227
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 29.065968619879364 27.41947147147147 1.0 0.0 0.013962030410766602
Node 228
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.006 )
( 11.1

( 15.837674425828505 12.093672247735288 0.34562075537705383 0.5378649117311112 4.499973297119141
Node 280
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 13.236263685695631 9.689132706066365 0.2967939079573046 0.6153389048530544 4.817154884338379
Node 281
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.051 )
( 15.625096731803703 11.874648865009378 0.34167070986601283 0.5463337917868643 4.511911392211914
Node 282
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 15.736425199564632 12.089040435644153 0.38653568087726975 0.5435105683592588 3.841761827468872
Node 283
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.008 )
( 14.228711117755937 10.720966089642774 0.3322406434926522 0.5915298287361029 3.531575918197632
Node 284
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.018 )
( 11.786815241540893 8.753036009658489 0.3247104510149777 0.661786158642332 4.015258073806763
Node 285
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 18.078478228246624 14.403032354311945 0.41334701456133666 0.482009

( 12.191256611978893 10.006184621286717 0.4173526978670997 0.6195413040134992 4.9757020473480225
Node 337
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.034 )
( 10.792717508450751 7.954122617809123 0.2965294620451011 0.662511790448761 4.811143159866333
Node 338
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.04 )
( 17.676464557319992 15.789853312151356 0.6741661986190779 0.44613705814432913 3.90556263923645
Node 339
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.035 )
( 11.208008059755096 8.616492211407952 0.35113999577265953 0.6479341348204125 4.541887521743774
Node 340
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.023 )
( 10.872300469755476 8.281664919876771 0.32502994114653355 0.6568336940499052 4.519922971725464
Node 341
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.036 )
( 11.725910020472183 9.081585555295236 0.35618415175807955 0.6286797639954587 3.591369867324829
Node 342
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 31.49303577844699 29.940420420420423 1.0 0.0 0.008976459503173828
Node 3

( 10.875748170023297 7.673170913518811 0.3435904068929063 0.578630373500777 4.15089225769043
Node 392
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.005 )
( 10.742740916726591 7.597374340057243 0.35873832214194235 0.584218925311069 4.739340543746948
Node 393
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.024 )
( 11.884585910877387 9.084975475743978 0.47602754629162625 0.5393788785210307 4.083095073699951
Node 394
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.014 )
( 10.53245841018803 7.919017948389869 0.4426780689105232 0.5901488398892312 3.792855739593506
Node 395
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.031 )
( 11.74179352364061 8.615765468929578 0.4213166611995267 0.5388595407861003 4.27856707572937
Node 396
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.021 )
( 10.890987176511313 8.228253144799664 0.4622945972329797 0.570934552535606 4.2456536293029785
Node 397
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.016 )
( 12.982720737121802 11.104950209576476 0.6972070249311855 0.4870553099535336

( 11.170095917518161 8.797913182161194 0.48869101921062885 0.560478309977746 3.7519726753234863
Node 447
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 10.139964312983134 7.801522980306256 0.43276107536950825 0.6012684756273587 3.9085850715637207
Node 448
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 25.45036589057148 23.538054054054058 1.0 0.0 0.008977413177490234
Node 449
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.054 )
( 10.680405258727992 7.919068941048356 0.38403151592920887 0.5805943167449992 4.871979475021362
Node 450
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 9.80401650954288 7.265802232536412 0.3772257172968544 0.6150264998118102 4.304464340209961
Node 451
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.018 )
( 11.356119209503314 9.075954339352542 0.5005830060391134 0.5572803410620877 3.950472831726074
Node 452
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.027 )
( 9.915222621295698 7.301340093995279 0.3543601206832903 0.6172101106152649 4.149879217147827
Node 453

( 10.283972484396958 7.50327749082033 0.28722902066649425 0.6655803702175749 4.073145151138306
Node 502
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.01 )
( 11.501706477288717 8.444343888541603 0.31540891701782453 0.6282973740610331 3.337080717086792
Node 503
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 31.098874622448687 29.391609609609606 1.0 0.0 0.009944438934326172
Node 504
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 31.22055887258133 29.52315315315314 1.0 0.0 0.007978439331054688
Node 505
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 14.0267295194957 10.386921334573792 0.3159139855977092 0.5532847358178613 3.7439939975738525
RMSE: 16.959452902711245, MAE: 14.312118231129547, MAPE: 0.5603329040037451, ACC: 0.4253030146912129
